In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import json
import pandas as pd
from pathlib import Path
from copy import deepcopy
from functools import partial

from bellek.qa.ablation import answer_question_standard as _answer_question_standard
from bellek.utils import set_seed, jprint
from bellek.musique.singlehop import benchmark

set_seed(89)

In [3]:
from tqdm.auto import tqdm
tqdm.pandas()

# Dataset

In [4]:
from bellek.musique.constants import ABLATION_RECORD_IDS

df = pd.read_json('../../data/generated/musique-evaluation/dataset.jsonl', orient='records', lines=True)

# Take ablation subset
df = df.set_index('id', drop=False).loc[ABLATION_RECORD_IDS].copy().reset_index(drop=True)

print(len(df))
df.head()

100


,id,paragraphs,question,question_decomposition,answer,answer_aliases,answerable,answers
0,2hop__575188_342798,"[{'idx': 0, 'title': 'Liliana Mumy', 'paragrap...",Who is the child of Mahmoud Mirza's father?,"[{'id': 575188, 'question': 'Mahmoud Mirza >> ...",Ahmad Shah Qajar,[Ahmad Shah Qajar],True,[Ahmad Shah Qajar]
1,2hop__731584_700117,"[{'idx': 0, 'title': 'KAPE', 'paragraph_text':...",In which county is the city to which KKVU is l...,"[{'id': 731584, 'question': 'KKVU >> licensed ...",Berrien County,[Berrien County],True,[Berrien County]
2,2hop__690412_526810,"[{'idx': 0, 'title': 'Cabramatta Creek', 'para...",For what river does the river on which Pa Sak ...,"[{'id': 690412, 'question': 'Pa Sak Jolasid Da...",Chao Phraya River,[Chao Phraya River],True,[Chao Phraya River]
3,2hop__263638_69048,"[{'idx': 0, 'title': 'Michael J. Barron', 'par...",Who is the Chief Judge of the Tebesa Nemine's ...,"[{'id': 263638, 'question': 'Tebesa Nemine >> ...",Honorable Justice Abiodun Smith,[Honorable Justice Abiodun Smith],True,[Honorable Justice Abiodun Smith]
4,2hop__142842_68489,"[{'idx': 0, 'title': 'Perfect Night: Live in L...",Who did the performer of Night Rocker play on ...,"[{'id': 142842, 'question': 'Which performer r...",Snapper Foster,[Snapper Foster],True,[Snapper Foster]


In [5]:
df['paragraphs'] = df['paragraphs'].map(lambda ps: [p for p in ps if p['is_supporting']])

# Definitions

In [6]:
def answer_question_standard(*args, **kwargs):
    return _answer_question_standard(*args, **kwargs).dict()

In [7]:
perfect_retrieval_func = lambda docs, query: [doc for doc in docs if doc['is_supporting']]

In [8]:
qa_func = answer_question_standard

In [9]:
from bellek.jerx.fewshot.llm import DEFAULT_JERX_SYSTEM_MESSAGE_FOR_LLAMA, DEFAULT_FEW_SHOT_EXAMPLE_MESSAGES
from bellek.jerx.fewshot.llm import make_kg_triplet_extract_fn

PREFIX_MESSAGES = [
    dict(role="system", content=DEFAULT_JERX_SYSTEM_MESSAGE_FOR_LLAMA),
    *DEFAULT_FEW_SHOT_EXAMPLE_MESSAGES,
]

In [10]:
default_completion_params = {
    "temperature": 0.1
}

def make_paragraph_replacer(model: str, prefix_messages: list[dict], completion_params: dict = default_completion_params):
    extract_kg_triplets = make_kg_triplet_extract_fn(model=model, prefix_messages=prefix_messages, completion_params=completion_params)

    def replace_paragraphs(row):
        new_paragraphs = []
        for p in row['paragraphs']:
            p = deepcopy(p) 
            triplets_str = '\n'.join(" | ".join(triplet) for triplet in extract_kg_triplets(p['paragraph_text']))
            p['paragraph_text'] = '\n'.join(["# Entity-relation-entity triplets", triplets_str])
            new_paragraphs.append(p)
        row['paragraphs'] = new_paragraphs
        return row
    
    return replace_paragraphs

# Experiments

In [11]:
N_RUNS = 1

results = []

## llama-zero-shot

In [12]:
# rp_zs = make_paragraph_replacer('llama-3-8b-tgi', PREFIX_MESSAGES[:1])

# for i in range(1, N_RUNS+1):
#     df_llama_zs = df.progress_apply(rp_zs, axis=1) 
#     df_llama_zs, scores = benchmark(df_llama_zs, qa_func, perfect_retrieval_func)
#     results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-zero-shot", "run": i})
#     jprint(scores)

## llama-few-shot

In [13]:
# rp_fs = make_paragraph_replacer('llama-3-8b-tgi', PREFIX_MESSAGES)

# for i in range(1, N_RUNS+1):
#     df_llama_fs = df.progress_apply(rp_fs, axis=1) 
#     df_llama_fs, scores = benchmark(df_llama_fs, qa_func, perfect_retrieval_func)
#     results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-few-shot", "run": i})
#     jprint(scores)

## llama-sft

In [22]:
rp_sft_zs = make_paragraph_replacer('llama-3-8b-sft-tgi', PREFIX_MESSAGES[:1])

for i in range(1, N_RUNS+1):
    df_llama_sft_zs = df.progress_apply(rp_sft_zs, axis=1) 
    df_llama_sft_zs, scores = benchmark(df_llama_sft_zs, qa_func, perfect_retrieval_func)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-sft-zs", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.48,
  "f1": 0.5464285714285714,
  "fuzzy_match": 0.56
}


In [15]:
rp_sft_fs = make_paragraph_replacer('llama-3-8b-sft-tgi', PREFIX_MESSAGES)

for i in range(1, N_RUNS+1):
    df_llama_sft_fs = df.progress_apply(rp_sft_fs, axis=1) 
    df_llama_sft_fs, scores = benchmark(df_llama_sft_fs, qa_func, perfect_retrieval_func)
    results.append({**scores, "retrieval": "groundtruth", "context": "triplets", "jerx": "llama-sft-fs", "run": i})
    jprint(scores)

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

{
  "exact_match": 0.46,
  "f1": 0.5494285714285714,
  "fuzzy_match": 0.55
}


# Report

In [16]:
pd.options.display.float_format = '{:,.3f}'.format

In [17]:
report_df = pd.DataFrame.from_records(results, columns=['jerx', 'run', 'exact_match', 'f1'])
report_df.rename(columns={'jerx': 'Model', 'exact_match': 'EM', 'f1': 'F1'}, inplace=True)

In [18]:
model_name_map = {
    'llama-zero-shot': 'Zero-shot prompted',
    'llama-few-shot': 'Few-shot prompted',
    'llama-sft-zs': 'Supervised fine-tuned (ZS)',
    'llama-sft-fs': 'Supervised fine-tuned (FS)',
}
report_df['Model'] = report_df['Model'].map(lambda x: model_name_map[x])

In [19]:
report_df

,Model,run,EM,F1
0,Supervised fine-tuned (FS),1,0.460,0.549


In [20]:
with open("ablation-jerx-llama-mhqa-results-all.tex", 'w') as f:
    f.write(report_df.to_latex(index=False, float_format='%.3f'))

In [21]:
agg_report_df = report_df[['Model', 'EM', 'F1']].groupby('Model').mean().loc[['Zero-shot prompted', 'Few-shot prompted', 'Supervised fine-tuned (ZS)', 'Supervised fine-tuned (FS)']].reset_index()
agg_report_df

KeyError: "['Zero-shot prompted', 'Few-shot prompted', 'Supervised fine-tuned (ZS)'] not in index"

In [ ]:
with open("ablation-jerx-llama-mhqa-results-agg.tex", 'w') as f:
    f.write(agg_report_df.to_latex(float_format='%.3f'))